<a href="https://colab.research.google.com/github/md-muqtadir-fuad/ml-models/blob/main/CNN_REGRESSION.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np

# Random seed for reproducibility
np.random.seed(42)

# Number of samples
n_samples = 1000

# Generate features (e.g., advertisement spend, season, store traffic, etc.)
ad_spend = np.random.uniform(1000, 5000, size=n_samples)  # Advertisement spend
season = np.random.choice([0, 1], size=n_samples)  # 0: Off-season, 1: Peak season
traffic = np.random.uniform(500, 5000, size=n_samples)  # Store traffic
discount = np.random.uniform(0, 1, size=n_samples)  # Discount applied (0-100%)

# Target: Sales = ad_spend * traffic * (1 + season factor) - discount factor
sales = ad_spend * (1 + 0.5 * season) * (traffic / 1000) - (discount * 500)

# Create a DataFrame
df = pd.DataFrame({
    'ad_spend': ad_spend,
    'season': season,
    'traffic': traffic,
    'discount': discount,
    'sales': sales
})

df.head()


,ad_spend,season,traffic,discount,sales
0,2498.160475,0,2835.868033,0.827519,6670.693973
1,4802.857226,1,2656.318449,0.764528,18754.613490
2,3927.975767,1,615.389296,0.573529,3339.086888
3,3394.633937,1,2035.615224,0.956047,9887.229212
4,1624.074562,1,2210.880285,0.200475,5285.714386


In [ ]:
X = df.drop('sales', axis=1)
y = df['sales']

In [ ]:
X.head(5)

,ad_spend,season,traffic,discount
0,2498.160475,0,2835.868033,0.827519
1,4802.857226,1,2656.318449,0.764528
2,3927.975767,1,615.389296,0.573529
3,3394.633937,1,2035.615224,0.956047
4,1624.074562,1,2210.880285,0.200475


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, Dense, Flatten, MaxPooling1D
#converting into numpy array
X_train = np.array(X_train_scaled)
X_test = np.array(X_test_scaled)

X_train_reshaped = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
X_test_reshaped = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)

model = Sequential([
    Conv1D(filters=32, kernel_size=3, activation='relu', input_shape=(X_train_reshaped.shape[1], 1)),
    MaxPooling1D(pool_size=1),
    Conv1D(filters=32, kernel_size=2, activation='relu'),
    Flatten(),
    Dense(64, activation='relu'),
    Dense(1)
])

model.compile(optimizer = 'adam', loss = 'mean_squared_error', metrics = ['mae'])

model.summary()

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d_7 (Conv1D)               │ (None, 2, 32)          │           128 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_2 (MaxPooling1D)  │ (None, 2, 32)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_8 (Conv1D)               │ (None, 1, 32)          │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_3 (Flatten)             │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 64)             │         2,112 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,385 (17.13 KB)

 Trainable params: 4,385 (17.13 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
history = model.fit(X_train_reshaped, y_train, epochs=50, batch_size=32, validation_split = .2)

Epoch 1/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 2s 15ms/step - loss: 143231408.0000 - mae: 9846.5723 - val_loss: 127122712.0000 - val_mae: 8992.9033
Epoch 2/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 157797184.0000 - mae: 10386.1855 - val_loss: 127093904.0000 - val_mae: 8991.3574
Epoch 3/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 155168160.0000 - mae: 10109.0840 - val_loss: 126994280.0000 - val_mae: 8986.1484
Epoch 4/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 161208992.0000 - mae: 10415.3857 - val_loss: 126700776.0000 - val_mae: 8971.2754
Epoch 5/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 133968224.0000 - mae: 9514.3857 - val_loss: 125963112.0000 - val_mae: 8935.0537
Epoch 6/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 144424240.0000 - mae: 9765.7676 - val_loss: 124346672.0000 - val_mae: 8858.2109
Epoch 7/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 132184368.0000 - mae: 9381.4346 - val_loss: 121285032.0000 - val_mae: 8715.7373
Epoch 8/50
20/20 ━━━━━━━━━━━━━

In [ ]:
!pip install keras-tuner

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 9.3 MB/s eta 0:00:00


In [ ]:
import keras_tuner as kt

def build_model(hp):
    model = Sequential([
        Conv1D(filters=hp.Int('filters_1', min_value=32, max_value=128, step=32), kernel_size=3, activation='relu', input_shape=(X_train_reshaped.shape[1], 1)),
        MaxPooling1D(pool_size=1),
        Conv1D(filters=hp.Int('filters_2', min_value=32, max_value=128, step=32), kernel_size=2, activation='relu'),
        Flatten(),
        Dense(hp.Int('dense_units', min_value=32, max_value=128, step=32), activation='relu'),
        Dense(1)
    ])
    model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])
    return model

tuner = kt.RandomSearch(
    build_model,
    objective='val_loss',
    max_trials=5,
    directory='my_dir',
    project_name='my_project'
)

tuner.search(X_train_reshaped, y_train, epochs=50, validation_data=(X_test_reshaped, y_test))

Trial 5 Complete [00h 00m 17s]
val_loss: 1793625.875

Best val_loss So Far: 1303025.5
Total elapsed time: 00h 01m 23s


In [ ]:
X_train_reshaped.shape[1]

4

In [ ]:
# Get the best model
best_model = tuner.get_best_models(num_models=1)[0]

# Evaluate the model on the test set
test_loss, test_mae = best_model.evaluate(X_test_reshaped, y_test)
print(f'Test Loss: {test_loss}, Test MAE: {test_mae}')


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.11/dist-packages/keras/src/saving/saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 18 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - loss: 1365944.6250 - mae: 936.1610  
Test Loss: 1303025.5, Test MAE: 901.236572265625


In [ ]:
# Make predictions on the test set
y_pred = best_model.predict(X_test_reshaped)

# Show a few predictions
print("Predictions:\n", y_pred[:5])


7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step 
Predictions:
 [[ 9841.22 ]
 [ 9433.165]
 [17334.717]
 [11433.996]
 [22091.576]]
